In [1]:
%ls data/plantseed
%matplotlib inline

preview/  test/  train/  train.zip  valid/


In [2]:
data_path = "data/plantseed/"

In [3]:
import numpy as np
import math
import cv2
import matplotlib
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19

from keras.optimizers import  Adam, RMSprop
from keras.models import Sequential, Model
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)



# Get data for training and validation
Import and init ImageData generator for future image processing from directory

In [4]:
train_imgs = ImageDataGenerator(
                                
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='nearest')
valid_imgs = ImageDataGenerator(
                                
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='nearest')
test_imgs = ImageDataGenerator()


In [5]:

def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2HSV)
    sensitivity = 38
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])
    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [6]:
img_size = 224 # because vgg16 input is 224,244,3 ; 3 here is for RGB 
batch_size = 64
img_batch_size =16
train_imgs_gen = train_imgs.flow_from_directory(data_path + "train", 
                                                target_size=(img_size,img_size), 
                                                batch_size=img_batch_size,
                                                color_mode = "rgb",
                                                class_mode='categorical',
                                                shuffle=True)

valid_imgs_gen = valid_imgs.flow_from_directory(data_path + "valid", 
                                                target_size=(img_size,img_size), 
                                                batch_size=img_batch_size,
                                                color_mode = "rgb",
                                                class_mode='categorical',
                                                shuffle=True)

test_imgs_gen = test_imgs.flow_from_directory(data_path + "test",
                                              target_size=(img_size,img_size), 
                                              batch_size=img_batch_size,
                                              color_mode = "rgb",
                                              class_mode=None,
                                              shuffle=False)
#model.load_weights('my_custom_vgg16_model_sgd_4.weight')



def train_imgs_gen_filter():
    res = []
    data = []

    for gen_data in train_imgs_gen:
        c = 0
        im_fix = []
        for x in gen_data[0]:
            z = sharpen_image(segment_plant(x))
            im_fix.append(z.tolist())
        data = gen_data[1]

        yield (np.array(im_fix,dtype=np.float32),np.array(data,dtype=np.float32))
    
def valid_imgs_gen_filter():

    res = []
    data = []

    for gen_data in valid_imgs_gen:
        c = 0    
        im_fix = []
        for x in gen_data[0]:
            z = sharpen_image(segment_plant(x))
            im_fix.append(z.tolist())
        data = gen_data[1]
        yield (np.array(im_fix,dtype=np.float32),np.array(data,dtype=np.float32))
    
        
def test_imgs_gen_filter():

    res = []
    data = []

    for gen_data in test_imgs_gen:
        c = 0
        im_fix = []
        for x in gen_data[0]:
            z = sharpen_image(segment_plant(x))
            im_fix.append(z.tolist())

        data = gen_data[1]
        yield (np.array(im_fix,dtype=np.float32), np.array(data,dtype=np.float32))

Found 4443 images belonging to 12 classes.
Found 307 images belonging to 12 classes.
Found 794 images belonging to 1 classes.


In [7]:
def plot_for_generator(generator):
    nb_rows = 4
    nb_cols = 4
    fig, axs = plt.subplots(nb_rows, nb_cols, figsize=(20, 20))
    c = 0
    for x in generator:
        axs[c % nb_rows, c // nb_cols].imshow(x.astype(np.uint8))
        c += 1    
#plot_for_generator(next(train_imgs_gen_filter())[0])

# from zf_unet_224_model import ZF_UNET_224, dice_coef_loss, dice_coef
# from keras.optimizers import Adam

# model_unet = ZF_UNET_224()
# model_unet.load_weights("zf_unet_224.h5") # optional
# optim = Adam()
# model_unet.compile(optimizer=optim, loss=dice_coef_loss, metrics=[dice_coef])

# model_unet.fit_generator(  train_imgs_gen, 
#                           samples_per_epoch= 4443,
#                           nb_epoch = 1,
#                           validation_data = valid_imgs_gen,
#                           nb_val_samples = 307
#                          )

# Load VGG16 keras model

In [8]:
vgg16 = VGG16()

vgg16.layers.pop()
for layer in vgg16.layers: 
    layer.trainable=False

last = vgg16.layers[-1].output
x = Dense(12, activation="softmax")(last)
model = Model(vgg16.input, x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [9]:
layers = model.layers
last_conv_idx = [idx for idx, layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers = layers[:last_conv_idx + 1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx + 1:]
fc_model = Sequential(fc_layers)
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
block5_pool (MaxPooling2D)       (None, 7, 7, 512)     0           block5_conv3[0][0]               
____________________________________________________________________________________________________
flatten (Flatten)                (None, 25088)         0           block5_pool[0][0]                
                                                                   block5_pool[0][0]                
____________________________________________________________________________________________________
fc1 (Dense)                      (None, 4096)          102764544   flatten[0][0]                    
                                                                   flatten[1][0]                    
___________________________________________________________________________________________

In [10]:
val_features = conv_model.predict_generator(valid_imgs_gen, valid_imgs_gen.nb_sample)

In [11]:
train_features = conv_model.predict_generator(train_imgs_gen, train_imgs_gen.nb_sample)

In [12]:
train_features.shape

(4443, 14, 14, 512)

In [13]:
def reweight(layer): return [o/2 for o in layer.get_weights()]

In [14]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [15]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(.5),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(.5),
        Dense(12, activation='softmax')
    ])
    model.summary()
#    for l1, l2 in zip(model.layers, fc_layers): 
#        print(l1, l2)
#        if type(l2) is MaxPooling2D or  type(l2) is Dense:
#            l1.set_weights(reweight(l2))
    model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
fc_model = get_fc_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_1 (MaxPooling2D)    (None, 7, 7, 512)     0           maxpooling2d_input_1[0][0]       
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 25088)         0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 4096)          102764544   flatten_1[0][0]                  
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 4096)          16384       dense_2[0][0]                    
___________________________________________________________________________________________

In [17]:
for layer in conv_model.layers: layer.trainable = False
for layer in fc_model.layers: layer.trainable = True

In [18]:
conv_model = Sequential(conv_layers)
conv_model.add(fc_model)

In [ ]:
conv_model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for count in range(1):
    conv_model.fit_generator(  train_imgs_gen, 
                          samples_per_epoch= train_imgs_gen.nb_sample,
                          nb_epoch = 7,
                          validation_data = valid_imgs_gen,
                          nb_val_samples = valid_imgs_gen.nb_sample
                         )
#model.load_weights(f'my_plantseed_vgg16_model_adam_6.weight')

Epoch 1/7
1248/4443 [=======>......................] - ETA: 90s - loss: 3.5905 - acc: 0.1218 

In [ ]:
conv_model.save_weights(f'my_plantseed_resnet50_clean.weight')
# from PIL import Image

# img = cv2.imread('./data/dogscats_new/test/unknown/333.jpg')
# img=cv2.resize(img,(224,224))
# img=np.array(img).reshape((224,224,3))
# plt.imshow(img.astype(np.uint8))
# img = img.reshape((1,) + img.shape)
# np.rint(model.predict(img))


In [ ]:
model_gen = conv_model.predict_generator(test_imgs_gen, test_imgs_gen.nb_sample)

In [ ]:
print(model_gen)
print(train_imgs_gen.class_indices)

In [ ]:
submission_file_name = 'submission_vgg19.csv'
filenames = test_imgs_gen.filenames
ids = np.array([f.replace("unknown/","") for f in filenames])

#ins = np.rint(model_gen)

names = []
resx = model_gen.argmax(axis=1)
print(resx)
z = train_imgs_gen.class_indices

res = dict((v,k) for k,v in z.items())
for x in resx:
    names.append(res.get(x))
print(names)

In [ ]:
print(len(ids), len(names))
subm = np.stack([ids,names], axis=1)
subm[:5]
np.savetxt(submission_file_name, subm, fmt='%s,%s', header='file,species', comments='')

In [ ]:
from IPython.display import FileLink
FileLink(submission_file_name)